In [14]:
import rioxarray
import geopandas
import os
import datetime
import pandas as pd
import numpy as np
from tqdm import tqdm

In [15]:
out_dir = "data/_grace_tiff"
os.makedirs(out_dir, exist_ok=True)
geodf = geopandas.read_file("data/aoi.geojson")
nc = rioxarray.open_rasterio("data/GRCTellus.JPL.200204_202202.GLO.RL06M.MSCNv02CRI.nc", decode_times=False)[0]
grace_start_date = datetime.datetime(2002, 1, 1)

In [20]:
nc = nc.rio.write_crs("EPSG:4326")
print("Extracting tifs...")
df = pd.DataFrame(columns = ["date", "value"])
for lwe in tqdm(nc["lwe_thickness"]):
    lwe = lwe.rio.clip(geodf.geometry.values, geodf.crs, all_touched=True)
    lwe = lwe.rio.write_crs("EPSG:4326")
    upscale_factor = 16
    new_width = lwe.rio.width * upscale_factor
    new_height = lwe.rio.height * upscale_factor

    lwe_upsampled = lwe.rio.reproject(
        lwe.rio.crs,
        shape=(new_height, new_width),
    )
    lwe_upsampled
    lwe_datetime = grace_start_date + datetime.timedelta(days=int(lwe.time.values))
    lwe_upsampled.values[lwe_upsampled.values < -99990] = np.nan
    mean = lwe_upsampled.mean(skipna=True)
    masked = lwe.to_masked_array()
    df = df.append({'date': lwe_datetime.strftime('%Y-%m') + "-01", 'value': masked.data}, ignore_index=True)

Extracting tifs...


100%|██████████| 206/206 [00:03<00:00, 60.40it/s]


In [21]:
df

,date,value
0,2002-04-01,"[[-99999.0, 6.828635107805798, 3.1278647095598..."
1,2002-05-01,"[[-99999.0, 4.69056193508014, 2.14384610714305..."
2,2002-08-01,"[[-99999.0, 1.9737695250682254, -1.03765383641..."
3,2002-09-01,"[[-99999.0, 0.18124379122407497, -3.1504822856..."
4,2002-10-01,"[[-99999.0, 0.4888633431839067, -3.44842383830..."
...,...,...
201,2021-10-01,"[[-99999.0, -8.133473995626613, -14.5749380667..."
202,2021-11-01,"[[-99999.0, -6.908211127148264, -13.0303585656..."
203,2021-12-01,"[[-99999.0, -4.684630748573696, -8.77763886511..."
204,2022-01-01,"[[-99999.0, -2.104737686864083, -6.90364717749..."


In [22]:
df.loc[df["date"] == "2012-01-01"]
df.iloc[111]["date"] = "2011-12-01"
df.iloc[144]["date"] = "2015-05-01"
df.to_pickle("data/grace_df.pickle")